In [22]:
import network
import utils
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [23]:
net = network.WaterDistributionNetwork("ctown.inp")
net.set_time_params(duration=3600, hydraulic_step=300) # duration=1h, hydstep=5min

In [24]:
status = [1.0, 1.0]
actuators_status_dict = {uid: status for uid in net.pumps.uid.append(net.valves.uid)}

At each timestep the status is chaged from 0 to 1 or viceversa. So, we expect to see the alternation on/off at each new timestep.

In [25]:
net.run(interactive=True, status_dict=actuators_status_dict)

In [26]:
net.df_links_report.iloc[:, net.df_links_report.columns.get_level_values(2) == 'status']

link        pumps                                                          \
id            PU1    PU2    PU3    PU4    PU5    PU6    PU7    PU8    PU9   
properties status status status status status status status status status   
00:00:00      1.0    1.0    0.0    1.0    0.0    0.0    1.0    1.0    0.0   
00:05:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:10:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:15:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:20:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:24:01      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:25:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:30:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:35:00      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
00:39:57      1.0    1.0    1.0    1.0    0.0    1.0    1.0    1.0    1.0   
00:40:00      1.0    1.0    1.0    1.0    0.0    1.0    1.0    1.0    1.0   
00:45:00      1.0    1.0    1.0    1.0    0.0    1.0    1.0    1.0    1.0   
00:45:20      1.0    1.0    1.0    1.0    0.0    1.0    1.0    1.0    1.0   
00:50:00      1.0    1.0    1.0    1.0    0.0    1.0    1.0    1.0    1.0   
00:55:00      1.0    1.0    1.0    1.0    0.0    1.0    1.0    1.0    1.0   
01:00:00      1.0    1.0    1.0    1.0    0.0    1.0    1.0    1.0    1.0   

link                     valves                       
id           PU10   PU11     v1    V45    V47     V2  
properties status status status status status status  
00:00:00      1.0    0.0    1.0    1.0    1.0    1.0  
00:05:00      1.0    1.0    1.0    1.0    1.0    1.0  
00:10:00      1.0    1.0    1.0    1.0    1.0    1.0  
00:15:00      1.0    1.0    1.0    1.0    1.0    1.0  
00:20:00      1.0    1.0    1.0    1.0    1.0    1.0  
00:24:01      1.0    0.0    1.0    1.0    1.0    1.0  
00:25:00      1.0    0.0    1.0    1.0    1.0    1.0  
00:30:00      1.0    0.0    1.0    1.0    1.0    1.0  
00:35:00      1.0    0.0    1.0    1.0    1.0    1.0  
00:39:57      1.0    0.0    1.0    1.0    1.0    1.0  
00:40:00      1.0    0.0    1.0    1.0    1.0    1.0  
00:45:00      1.0    0.0    1.0    1.0    1.0    1.0  
00:45:20      1.0    0.0    1.0    1.0    1.0    1.0  
00:50:00      1.0    0.0    1.0    1.0    1.0    1.0  
00:55:00      1.0    0.0    1.0    1.0    1.0    1.0  
01:00:00      1.0    0.0    1.0    1.0    1.0    1.0

Let's check if PU8 respects the control rules of tank T5

In [27]:
net.df_nodes_report['tanks', 'T5', 'pressure']

00:00:00    1.046840
00:05:00    1.117495
00:10:00    1.188070
00:15:00    1.258568
00:20:00    1.315137
00:24:01    1.330120
00:25:00    1.406282
00:30:00    1.482354
00:35:00    1.557575
00:39:57    1.558334
00:40:00    1.634228
00:45:00    1.639282
00:45:20    1.710296
00:50:00    1.786300
00:55:00    1.862216
01:00:00    1.862216
Name: (tanks, T5, pressure), dtype: float64

In [71]:
net.pumps['PU1'].results['flow'][-1]

78.74488885890791

In [72]:
pump_dict = {key: net.pumps['PU1'].results[key][-1] for key in ['status', 'flow']}
pump_dict

{'status': 1.0, 'flow': 78.74488885890791}

In [83]:
last_values = pd.Series()

for uid in net.pumps.results.index.append(net.valves.results.index):
    links_dict = {key: net.links[uid].results[key][-1] for key in ['status', 'flow']}
    last_values[uid] = links_dict

for uid in net.tanks.results.index.append(net.junctions.results.index):
    nodes_dict = {key: net.nodes[uid].results[key][-1] for key in ['pressure']}
    last_values[uid] = nodes_dict
    
last_values

PU1      {'status': 1.0, 'flow': 78.74488885890791}
PU2        {'status': 1.0, 'flow': 78.758886094791}
PU3      {'status': 1.0, 'flow': 78.74492945353111}
PU4     {'status': 1.0, 'flow': 33.048078664173914}
PU5                    {'status': 0.0, 'flow': 0.0}
                           ...                     
J306                {'pressure': 98.14923554479816}
J307               {'pressure': 19.151136905156537}
J309                {'pressure': 19.15113690515768}
J317                {'pressure': 73.39425512927197}
J323                 {'pressure': 73.3942551292704}
Length: 410, dtype: object